# Check angles 

- author Sylvie Dagoret-Campagne
- creation date 2024-10-01 : version v4
- last update : 2024-10-01 :
- affiliation : IJCLab
- Kernel @usdf **w_2024_38*
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figsCheckAngles"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
version_results = "v4"
legendtag = {"v1" : "old v3.1.0",
            "v2" : "v3.1.0-PWV<10mm",
            "v3" : "v3.1.0-PWV<15mm",
            "v4" : "v3.1.0-extended"}

In [ ]:
atmfilenamesdict = {"v1" : "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy",
                    "v2" : "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy",
                    "v3" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy",
                    "v4" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z_spectrfullextend.npy"}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
list(df_spec.columns)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

#### Series on spec

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title("nb obs per night")
ax.set_xlabel("Nobs/night")

### Add the Time in pd.datetime

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

#df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
#ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
#ax.set_title("Precipitable water vapor measured by holo vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)
ax.tick_params(axis='x', labelrotation=90)

plt.tight_layout()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt"].values
    y = df_night_pwv_curve["PWV [mm]_x"].values
    yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

### Plot night by night

In [ ]:

# loop on nights
for night in all_selected_nights:
    #select the night
    df_spec_night = df_spec[df_spec["nightObs"] == night]
    
    #select the variables
    df_night_curve = df_spec_night[["Time","ex_skyangle","ex_azimuth","ex_zenith"]]

    tmin = df_night_curve["Time"].min()

    # convert in hours
    df_night_curve["dt"] = (df_night_curve["Time"] - tmin).dt.total_seconds()/3600.
    
   

    

    N= len(df_night_curve)

    if N>10:
        
        
        # plot
      
        
        fig = plt.figure(figsize=(16,6))
        gs = GridSpec(1, 2,  width_ratios=[2,1],figure=fig)

        ax = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1],sharey=ax)
        
        leg=ax.get_legend()
        leg2=ax2.get_legend()
      
        
        # left figure
        df_night_curve.plot(x="Time",y="ex_skyangle",ax=ax,marker='+',c="r",lw=0.0,grid=True,legend=leg,label="sky-angle",ms=10)
        #ax.errorbar(x=df_night_pwv_curve.Time, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt=".",color="r",ecolor="k")
        
        ax.legend(loc="upper right")
        ax.set_ylabel("Sky angle")
        ax.set_xlabel("time")
        ax.xaxis.set_major_formatter(date_form)
        #ax.set_ylim(0.,15.)
        #ax.text(0.03, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        #verticalalignment='top', bbox=props)
        #ax.text(0.35, 0.95, textstr3, transform=ax.transAxes, fontsize=14,
        #verticalalignment='top', bbox=props)
        #ax.set_title(f"night {night}")

        # right figure
        df_night_curve.plot(x="dt",y="ex_skyangle",ax=ax2,marker='+',c="r",lw=0.0,grid=True,legend=leg,label="sky angle",ms=10)
        #ax2.errorbar(x=df_night_curve.dt, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",ms=10,label=tag)
        ax2.set_xlabel("hours")
        #ax2.set_ylim(0.,15.)
        ax2.grid()
        ax2.legend(loc="upper right")
        #ax2.plot(xfit,yfit,"k-")
        #ax2.text(0.45, 0.95, textstr2, transform=ax2.transAxes, fontsize=14,
        #verticalalignment='top', bbox=props)
        ax2.set_title(f"night {night}")
        
        plt.tight_layout()
        figname =f"{pathfigs}/pwv_per_night_{night}"+figtype
        plt.savefig(figname)
        plt.show()

       

In [ ]:
assert False